**Derived Tables (table subqueries)**

Derived tables are evaluated dynamically and are different from physical tables. It gives you the data upfront and can avoid many joins.

The following query shows the syntax derived tables. You have a subquery following the FROM clause within the parenthesis.  You end the parenthesis with alias. You end the parenthesis with alias. It is different from subqueries in that subqueries are embedded either in SELECT or WHERE clause of the outer query.

Proposition: Return information about customers from USA.

In [ ]:
Use Northwinds2022TSQLV7

SELECT *
FROM (SELECT customerId, customerCompanyName
      FROM Sales.Customer
      WHERE customerCountry = N'USA') AS USACusts;

**3 Requirements for a Valid Table Expression**

1\. Order is not guaranteed

2\. All columns must have names

3\. All column names must be unique

**Query with a Dervied Table using Inline Aliasing Form**

In regular queries, you cannot refer to the column alias in the SELECT clause. In dervied tables, you can.

Proposition: Return a count of distinct customers and the order year.

In [1]:
Use Northwinds2022TSQLV7

--Notice that orderyear is a column alias from the inner query. It is used in the outer query's SELECT and GROUP BY Clauses.
SELECT orderyear, COUNT(DISTINCT customerId) AS numcusts
FROM (SELECT YEAR(orderdate) AS orderyear, customerId
      FROM Sales.[Order]) AS D
GROUP BY orderyear;


--In a regular query, you would have had to this:
SELECT YEAR(orderdate) AS orderyear, COUNT(DISTINCT customerId) AS numcusts
FROM Sales.[Order]
GROUP BY YEAR(orderdate);

(3 rows affected)

(3 rows affected)

Total execution time: 00:00:00.061

orderyear,numcusts
2014,67
2015,86
2016,81


orderyear,numcusts
2014,67
2015,86
2016,81


**Query with Derived Table Using External Aliasing**

Proposition: Return a count of distinct customers and the order year.

It is more dfficult to debug.

In [2]:
Use Northwinds2022TSQLV7
SELECT orderyear, COUNT(DISTINCT custid) AS numcusts
FROM (SELECT YEAR(orderdate), customerId
      FROM Sales.[Order]) AS D(orderyear, custid)
GROUP BY orderyear;


(3 rows affected)

Total execution time: 00:00:00.008

orderyear,numcusts
2014,67
2015,86
2016,81


**Using Arguments**

Like local variables and input parameters to a routine procedure.

Proposition: Return distinct number of customers and year that employeeID 3 helped.

In [3]:
Use Northwinds2022TSQLV7
DECLARE @empid AS INT = 3;

SELECT orderyear, COUNT(DISTINCT customerId) AS numcusts
FROM (SELECT YEAR(orderdate) AS orderyear, customerId
      FROM Sales.[Order]
      WHERE EmployeeId = @empid) AS D
GROUP BY orderyear;


(3 rows affected)

Total execution time: 00:00:00.020

orderyear,numcusts
2014,16
2015,46
2016,30


**Nesting**

Tends to complicate the code and reduces readability.

Proposition: Return order years and the number of customers handled in each year only for years in which more than 70 customers were handled

In [4]:
Use Northwinds2022TSQLV7
SELECT orderyear, numcusts
FROM (SELECT orderyear, COUNT(DISTINCT CustomerId) AS numcusts
      FROM (SELECT YEAR(orderdate) AS orderyear, CustomerId
            FROM Sales.[Order]) AS D1
      GROUP BY orderyear) AS D2
WHERE numcusts > 70;

--The same result can be obtained by the following simpler, easier to read query:

SELECT YEAR(orderdate) AS orderyear, COUNT(DISTINCT CustomerId) AS numcusts
FROM Sales.[Order]
GROUP BY YEAR(orderdate)
HAVING COUNT(DISTINCT CustomerId) > 70;

(2 rows affected)

(2 rows affected)

Total execution time: 00:00:00.020

orderyear,numcusts
2015,86
2016,81


orderyear,numcusts
2015,86
2016,81


**Multiple References**

This query joins two derived tables that are based on the same query. The join condition, Cur.orderyear = prv.orderyear+1 ensures that each year from the first derived table matches the previous year of the second.

Proposition: Determine the amount of growth by calculating the difference in number of customers served in two consecutive years.

In [5]:
Use Northwinds2022TSQLV7
SELECT Cur.orderyear, 
  Cur.numcusts AS curnumcusts, Prv.numcusts AS prvnumcusts,
  Cur.numcusts - Prv.numcusts AS growth
FROM (SELECT YEAR(orderdate) AS orderyear,
        COUNT(DISTINCT CustomerId) AS numcusts
      FROM Sales.[Order]
      GROUP BY YEAR(orderdate)) AS Cur
  LEFT OUTER JOIN
     (SELECT YEAR(orderdate) AS orderyear,
        COUNT(DISTINCT CustomerId) AS numcusts
      FROM Sales.[Order]
      GROUP BY YEAR(orderdate)) AS Prv
    ON Cur.orderyear = Prv.orderyear + 1;

--Because we cannot refer to multiple instances of the same dervied table in the same join forces you to maintain multple copies of the same query definition. This is lengthy and prone to errors.


(3 rows affected)

Total execution time: 00:00:00.020

orderyear,curnumcusts,prvnumcusts,growth
2014,67,NULL,NULL
2015,86,67,19
2016,81,86,-5


**Common Table Expressions (CTE)**

  

CTE has more advantages than the derived tables. It starts with the WITH statement. 

  

Proposition: Return all customers from the US.

In [7]:
Use Northwinds2022TSQLV7;
WITH USACusts AS
(
  SELECT CustomerId, CustomerCompanyname
  FROM Sales.Customer
  WHERE CustomerCountry = N'USA'
)
SELECT * FROM USACusts;

--The table name comes before AS. The CTE is defined first then the outer query is written afterwards. 

(13 rows affected)

Total execution time: 00:00:00.013

CustomerId,CustomerCompanyname
32,Customer YSIQX
36,Customer LVJSO
43,Customer UISOJ
45,Customer QXPPT
48,Customer DVFMB
55,Customer KZQZT
65,Customer NYUHS
71,Customer LCOUJ
75,Customer XOJYP
77,Customer LCYBZ


**CTE: Assigning Column Aliases**

Proposition: Return a count of distinct customers and the order year.

In [8]:
Use Northwinds2022TSQLV7;
-- Inline column aliasing
WITH C AS
(
  SELECT YEAR(orderdate) AS orderyear, CustomerId
  FROM Sales.[Order]
)
SELECT orderyear, COUNT(DISTINCT CustomerId) AS numcusts
FROM C
GROUP BY orderyear;

-- External column aliasing
WITH C(orderyear, custid) AS
(
  SELECT YEAR(orderdate), CustomerId
  FROM Sales.[Order]
)
SELECT orderyear, COUNT(DISTINCT custid) AS numcusts
FROM C
GROUP BY orderyear;


(3 rows affected)

(3 rows affected)

Total execution time: 00:00:00.012

orderyear,numcusts
2014,67
2015,86
2016,81


orderyear,numcusts
2014,67
2015,86
2016,81


**CTE: Using Arguments**

Proposition: Return distinct number of customers and year that employeeID 3 helped.

In [9]:
Use Northwinds2022TSQLV7;
DECLARE @empid AS INT = 3;
WITH C AS
(
  SELECT YEAR(orderdate) AS orderyear, CustomerId
  FROM Sales.[Order]
  WHERE EmployeeId = @empid
)
SELECT orderyear, COUNT(DISTINCT CustomerId) AS numcusts
FROM C
GROUP BY orderyear;

(3 rows affected)

Total execution time: 00:00:00.008

orderyear,numcusts
2014,16
2015,46
2016,30


**Defining Multiple CTEs**

You do not nest the CTEs. You separate them by commas. Each CTE can refer to all previous CTE and the outer query can refer to all CTEs. This improves readability and maintainability

Proposition: Return order years and the number of customers handled in each year only for years in which more than 70 customers were handled

In [10]:
Use Northwinds2022TSQLV7;
WITH C1 AS
(
  SELECT YEAR(orderdate) AS orderyear, CustomerId
  FROM Sales.[Order]
),
C2 AS
(
  SELECT orderyear, COUNT(DISTINCT CustomerId) AS numcusts
  FROM C1
  GROUP BY orderyear
)
SELECT orderyear, numcusts
FROM C2
WHERE numcusts > 70;

(2 rows affected)

Total execution time: 00:00:00.008

orderyear,numcusts
2015,86
2016,81


**CTE: Multiple References**

Once CTE is created, you can refer to it multple times.

Proposition: Determine the amount of growth by calculating the difference in number of customers served in two consecutive years.

In [11]:
Use Northwinds2022TSQLV7;
WITH YearlyCount AS
(
  SELECT YEAR(orderdate) AS orderyear,
    COUNT(DISTINCT CustomerId) AS numcusts
  FROM Sales.[Order]
  GROUP BY YEAR(orderdate)
)
SELECT Cur.orderyear, 
  Cur.numcusts AS curnumcusts, Prv.numcusts AS prvnumcusts,
  Cur.numcusts - Prv.numcusts AS growth
FROM YearlyCount AS Cur
  LEFT OUTER JOIN YearlyCount AS Prv
    ON Cur.orderyear = Prv.orderyear + 1;

(3 rows affected)

Total execution time: 00:00:00.016

orderyear,curnumcusts,prvnumcusts,growth
2014,67,NULL,NULL
2015,86,67,19
2016,81,86,-5


**Recursive CTEs**

It is like a for-loop. You union anchor member with a recursive member.  Anchor member is a valid relational result. Recursive member defines the termination condition.

Proposition: Return employee 2 and all the employee's subordinates.

In [12]:
Use Northwinds2022TSQLV7;
WITH EmpsCTE AS
(
  SELECT EmployeeId, EmployeeManagerId, Employeefirstname, EmployeeLastname
  FROM HumanResources.Employee
  WHERE EmployeeId = 2
  
  UNION ALL
  
  SELECT C.EmployeeId, C.EmployeeManagerId, C.Employeefirstname, C.EmployeeLastname
  FROM EmpsCTE AS P
    INNER JOIN HumanResources.Employee AS C
      ON C.EmployeeManagerId = P.EmployeeId
)
SELECT EmployeeId, EmployeeManagerId, Employeefirstname, EmployeeLastname
FROM EmpsCTE;


(8 rows affected)

Total execution time: 00:00:00.014

EmployeeId,EmployeeManagerId,Employeefirstname,EmployeeLastname
2,1,Don,Funk
3,2,Judy,Lew
5,2,Sven,Mortensen
6,5,Paul,Suurs
7,5,Russell,King
9,5,Patricia,Doyle
4,3,Yael,Peled
8,3,Maria,Cameron


**Views**

They are stored as permanent objects in the databse, making them reusable. You can manage access.

Proposition: Return all customers from United States

In [13]:
Use Northwinds2022TSQLV7;
-- Creating USACusts View
DROP VIEW IF EXISTS Sales.USACusts;
GO
CREATE VIEW Sales.USACusts
AS

SELECT
  CustomerId, Customercompanyname, Customercontactname, Customercontacttitle, Customeraddress,
  Customercity, CustomerRegion, CustomerPostalcode, CustomerCountry, CustomerPhoneNumber, CustomerFaxNumber
FROM Sales.Customer
WHERE CustomerCountry = N'USA';
GO

--Using stored views
SELECT Customerid, CustomerCompanyname
FROM Sales.USACusts;
GO

Commands completed successfully.

Commands completed successfully.

(13 rows affected)

Total execution time: 00:00:00.026

Customerid,CustomerCompanyname
32,Customer YSIQX
36,Customer LVJSO
43,Customer UISOJ
45,Customer QXPPT
48,Customer DVFMB
55,Customer KZQZT
65,Customer NYUHS
71,Customer LCOUJ
75,Customer XOJYP
77,Customer LCYBZ


**ORDER BY in a View is not Allowed**

Proposition: Return all customers from United States

In [ ]:
--Instead, use ORDER BY in the outer query.
Use Northwinds2022TSQLV7;
SELECT CustomerId, Customercompanyname, Customerregion
FROM Sales.USACusts
ORDER BY Customerregion;

--You can create the view with ORDER BY by adding TOP or OFFSET FETCH. But the order is not guaranteed.


**View Options: ENCRYPTION**

The obfuscated text is not directly visible to users through any of the catalog objects- only to privileged users.

Proposition: Return USA customers

In [14]:
--The following meta data function is available becauses this view was created without encryption. It displays the definition of the view.
SELECT OBJECT_DEFINITION(OBJECT_ID('Sales.USACusts'));
GO

--now, adding encryption
ALTER VIEW Sales.USACusts WITH ENCRYPTION
AS

SELECT
  CustomerId, Customercompanyname, Customercontactname, Customercontacttitle, Customeraddress,
  Customercity, CustomerRegion, CustomerPostalcode, CustomerCountry, CustomerPhoneNumber, CustomerFaxNumber
FROM Sales.Customer
WHERE CustomerCountry = N'USA';
GO

--Try running the same function. It won't work. It only displays NULL.
SELECT OBJECT_DEFINITION(OBJECT_ID('Sales.USACusts'));

EXEC sp_helptext 'Sales.USACusts';--This will explain that the view is encrypted.

(1 row affected)

Commands completed successfully.

(1 row affected)

The text for object 'Sales.USACusts' is encrypted.

Total execution time: 00:00:00.127

(No column name)
"CREATE VIEW Sales.USACusts AS SELECT CustomerId, Customercompanyname, Customercontactname, Customercontacttitle, Customeraddress, Customercity, CustomerRegion, CustomerPostalcode, CustomerCountry, CustomerPhoneNumber, CustomerFaxNumber FROM Sales.Customer WHERE CustomerCountry = N'USA';"


(No column name)
NULL


**View Options: SCHEMABINDING**

It binds the schema of referenced objects and columns to the schema of the referencing object. It indicates that referenced objects cannot be dropped or altered. This is a good practice.

Proposition: Return USA customers

In [15]:
ALTER VIEW Sales.USACusts WITH SCHEMABINDING
AS

SELECT
  CustomerId, Customercompanyname, Customercontactname, Customercontacttitle, Customeraddress,
  Customercity, CustomerRegion, CustomerPostalcode, CustomerCountry, CustomerPhoneNumber, CustomerFaxNumber
FROM Sales.Customer
WHERE CustomerCountry = N'USA';
GO

--Now, you wont be able to drop or alter the view.
ALTER TABLE Sales.Customers DROP COLUMN address;

Commands completed successfully.

: Msg 4902, Level 16, State 1, Line 12
Cannot find the object "Sales.Customers" because it does not exist or you do not have permissions.

Total execution time: 00:00:00.024

**View Options: CHECK OPTIONS**

It prevents modifications through the view that conflict with the view's filter.

Proposition: Return USA customers

In [16]:
use Northwinds2022TSQLV7;
-- Notice that you can insert a row through the view
INSERT INTO Sales.USACusts(
  Customercompanyname, Customercontactname, Customercontacttitle, Customeraddress,
  Customercity, CustomerRegion, CustomerPostalcode, CustomerCountry, CustomerPhoneNumber, CustomerFaxNumber)
 VALUES(
  N'Customer ABCDE', N'Contact ABCDE', N'Title ABCDE', N'Address ABCDE',
  N'London', NULL, N'12345', N'UK', N'012-3456789', N'012-3456789');

-- But when you query the view, you won't see it because you inserted a customer from UK, not USA.
SELECT CustomerId, Customercompanyname, CustomerCountry
FROM Sales.USACusts
WHERE CustomerCompanyname = N'Customer ABCDE';

-- You can see it in the table, though
SELECT CustomerId, Customercompanyname, CustomerCountry
FROM Sales.Customer
WHERE CustomerCompanyname = N'Customer ABCDE';
GO

-- Add CHECK OPTION to the View
ALTER VIEW Sales.USACusts WITH SCHEMABINDING
AS

SELECT
  CustomerId, Customercompanyname, Customercontactname, Customercontacttitle, Customeraddress,
  Customercity, CustomerRegion, CustomerPostalcode, CustomerCountry, CustomerPhoneNumber, CustomerFaxNumber
FROM Sales.Customer
WHERE CustomerCountry = N'USA'
WITH CHECK OPTION;
GO

-- Notice that you can't insert a row through the view because the country should be from USA
/*
INSERT INTO Sales.USACusts(
  companyname, contactname, contacttitle, address,
  city, region, postalcode, country, phone, fax)
 VALUES(
  N'Customer FGHIJ', N'Contact FGHIJ', N'Title FGHIJ', N'Address FGHIJ',
  N'London', NULL, N'12345', N'UK', N'012-3456789', N'012-3456789');
*/
GO

-- Cleanup
DELETE FROM Sales.Customer
WHERE CustomerId > 91;

DROP VIEW IF EXISTS Sales.USACusts;
GO

(1 row affected)

(0 rows affected)

(1 row affected)

Commands completed successfully.

Commands completed successfully.

(1 row affected)

Total execution time: 00:00:00.063

CustomerId,Customercompanyname,CustomerCountry


CustomerId,Customercompanyname,CustomerCountry
1092,Customer ABCDE,UK


**Inline <span style="color: #569cd6;">Table</span><span style="color: #d4d4d4;">-</span>Valued Functions (TVF)**

Reusable table expressions that support input parameters. They are like views with parameters.

Proposition: Get customer orders. Customer ID 1

In [17]:
use Northwinds2022TSQLV7;
--Creating function
DROP FUNCTION IF EXISTS dbo.GetCustOrders;
GO
CREATE FUNCTION dbo.GetCustOrders
    (@cid AS INT) RETURNS TABLE
AS
RETURN
    SELECT orderid, CustomerId, EmployeeId, orderdate, requireddate,
        shipTodate, shipperid, freight, shipToname, shipToaddress, shipTocity,
        shipToregion, shipTopostalcode, shipTocountry
    FROM Sales.[Order]
    WHERE CustomerId = @cid;
GO

--using the function
SELECT orderid, CustomerId
FROM dbo.GetCustOrders(1) AS O;

--You can also perform joins with TVF
SELECT O.orderid, O.CustomerId, OD.productid, OD.Quantity
FROM dbo.GetCustOrders(1) AS O
  INNER JOIN Sales.OrderDetail AS OD
    ON O.orderid = OD.orderid;
GO

-- Cleanup
DROP FUNCTION IF EXISTS dbo.GetCustOrders;

Commands completed successfully.

Commands completed successfully.

(6 rows affected)

(12 rows affected)

Commands completed successfully.

Total execution time: 00:00:00.040

orderid,CustomerId
10643,1
10692,1
10702,1
10835,1
10952,1
11011,1


orderid,CustomerId,productid,Quantity
10643,1,28,15
10643,1,39,21
10643,1,46,2
10692,1,63,20
10702,1,3,6
10702,1,76,15
10835,1,59,15
10835,1,77,2
10952,1,6,16
10952,1,28,2


CROSS APPLY

Apply operator operates on two input tables. Cross apply implements only one logical-query processing phase. It appIies the right table to each row from the left table and produces a result table with the unified result sets. Only returns matching rows.   

Proposition: Return a cartesian product of the employeeID and shipper Id. Notice the cross join and cross apply return the same results.

In [18]:
use Northwinds2022TSQLV7;
SELECT S.shipperid, E.employeeId
FROM Sales.Shipper AS S
  CROSS JOIN HumanResources.Employee AS E;

SELECT S.shipperid, E.employeeid
FROM Sales.Shipper AS S
  CROSS APPLY HumanResources.Employee AS E;

(27 rows affected)

(27 rows affected)

Total execution time: 00:00:00.015

shipperid,employeeId
1,1
1,2
1,3
1,4
1,5
1,6
1,7
1,8
1,9
2,1


shipperid,employeeid
1,1
1,2
1,3
1,4
1,5
1,6
1,7
1,8
1,9
2,1


CROSS APPLY 

Proposition: Return 3 most recent orders for each customer. 

Cross apply is like inner join except there is some order for cross apply. With apply, left side is evaluated first, and the right side is evaluated per row from the left. It is like correlated derived table. The right table expression (a dervied table) is applied to each row from the Customers table.

In [19]:
use Northwinds2022TSQLV7;
SELECT C.CustomerId, A.orderid, A.orderdate
FROM Sales.Customer AS C
  CROSS APPLY
    (SELECT TOP (3) orderid, EmployeeId, orderdate, requireddate 
     FROM Sales.[Order] AS O
     WHERE O.CustomerId = C.CustomerId
     ORDER BY orderdate DESC, orderid DESC) AS A;

--You can use order by as well as offset fetch.
SELECT C.CustomerId, A.orderid, A.orderdate
FROM Sales.Customer AS C
  CROSS APPLY
    (SELECT orderid, EmployeeId, orderdate, requireddate 
     FROM Sales.[Order] AS O
     WHERE O.CustomerId = C.CustomerId
     ORDER BY orderdate DESC, orderid DESC
     OFFSET 0 ROWS FETCH NEXT 3 ROWS ONLY) AS A;

(263 rows affected)

(263 rows affected)

Total execution time: 00:00:00.036

CustomerId,orderid,orderdate
1,11011,2016-04-09
1,10952,2016-03-16
1,10835,2016-01-15
2,10926,2016-03-04
2,10759,2015-11-28
2,10625,2015-08-08
3,10856,2016-01-28
3,10682,2015-09-25
3,10677,2015-09-22
4,11016,2016-04-10


CustomerId,orderid,orderdate
1,11011,2016-04-09
1,10952,2016-03-16
1,10835,2016-01-15
2,10926,2016-03-04
2,10759,2015-11-28
2,10625,2015-08-08
3,10856,2016-01-28
3,10682,2015-09-25
3,10677,2015-09-22
4,11016,2016-04-10


OUTER APPLY

Outer apply is like the left outer join. It performs two-query processing phases. It returns the matching and nonmatching rows.   

Proposition: Return 3 most recent orders from each customer, preserve customers

In [20]:
use Northwinds2022TSQLV7;
-- 3 most recent orders for each customer, preserve all customers
SELECT C.CustomerId, A.orderid, A.orderdate
FROM Sales.Customer AS C
  OUTER APPLY
    (SELECT TOP (3) orderid, EmployeeId, orderdate, requireddate 
     FROM Sales.[Order] AS O
     WHERE O.CustomerId = C.CustomerId
     ORDER BY orderdate DESC, orderid DESC) AS A;

-- Customer 22 and 57 are included.

(265 rows affected)

Total execution time: 00:00:00.025

CustomerId,orderid,orderdate
1,11011,2016-04-09
1,10952,2016-03-16
1,10835,2016-01-15
2,10926,2016-03-04
2,10759,2015-11-28
2,10625,2015-08-08
3,10856,2016-01-28
3,10682,2015-09-25
3,10677,2015-09-22
4,11016,2016-04-10


Using Cross Apply with inline function.

Proposition: Return 3 most recent orders for each customer.

In [21]:
use Northwinds2022TSQLV7;
-- Creation Script for the Function TopOrders
DROP FUNCTION IF EXISTS dbo.TopOrders;
GO
CREATE FUNCTION dbo.TopOrders
  (@custid AS INT, @n AS INT)
  RETURNS TABLE
AS
RETURN
  SELECT TOP (@n) orderid, EmployeeId, orderdate, requireddate 
  FROM Sales.[Order]
  WHERE CustomerId = @custid
  ORDER BY orderdate DESC, orderid DESC;
GO

SELECT
  C.CustomerId, C.CustomerCompanyname,
  A.orderid, A.EmployeeId, A.orderdate, A.requireddate 
FROM Sales.Customer AS C
  CROSS APPLY dbo.TopOrders(C.CustomerId, 3) AS A;


Commands completed successfully.

Commands completed successfully.

(263 rows affected)

Total execution time: 00:00:00.043

CustomerId,CustomerCompanyname,orderid,EmployeeId,orderdate,requireddate
1,Customer NRZBB,11011,3,2016-04-09,2016-05-07
1,Customer NRZBB,10952,1,2016-03-16,2016-04-27
1,Customer NRZBB,10835,1,2016-01-15,2016-02-12
2,Customer MLTDN,10926,4,2016-03-04,2016-04-01
2,Customer MLTDN,10759,3,2015-11-28,2015-12-26
2,Customer MLTDN,10625,3,2015-08-08,2015-09-05
3,Customer KBUDE,10856,3,2016-01-28,2016-02-25
3,Customer KBUDE,10682,3,2015-09-25,2015-10-23
3,Customer KBUDE,10677,1,2015-09-22,2015-10-20
4,Customer HFBZG,11016,9,2016-04-10,2016-05-08
